# 🌍 DeMark-World — Auto Watermark Remover

**Universal AI Video Watermark Removal** — Sora, Runway, Veo, Pika, Kling, Luma & more.

> ⚠️ **Before you start:** Enable GPU via *Runtime → Change runtime type → T4 GPU*

### How this works
- **First time only (Step 1):** Installs everything into your Google Drive — takes ~5 min but you only ever do this once.
- **Every session after (Step 2):** Mounts Drive and loads from there — boots in ~30 seconds, no downloads.

**Repo:** https://github.com/linkedlist771/DeMark-World | **License:** MIT

## Step 1 — One-Time Install to Google Drive

**Run this only once.** Everything (repo, models, Python packages) is saved to your Drive under `Figuro/sora-watermark-remover/app/`.

*After this completes, you never need to run it again — even after Colab disconnects.*

In [1]:
#@title 💾 Step 1: Install to Personal Drive { display-mode: "form" }
app_drive_path = "google-colab/sora-watermark-remover"  #@param {type:"string"}

import os, sys, json, time, shutil
from pathlib import Path
from google.colab import drive

# 1. Mount
drive.mount('/content/drive', force_remount=False)

# 2. Paths
MY_DRIVE    = '/content/drive/MyDrive'
APP_DIR     = Path(MY_DRIVE) / app_drive_path / 'app'
REPO_DIR    = APP_DIR / 'DeMark-World'
MARKER_FILE = APP_DIR / '.install_complete'

if MARKER_FILE.exists():
    print(f"✅ App already installed at: {APP_DIR}")
else:
    print(f"🚀 Starting Installation...")
    APP_DIR.mkdir(parents=True, exist_ok=True)

    # Install system deps
    !apt-get update -qq && apt-get install -y ffmpeg -qq
    !curl -LsSf https://astral.sh/uv/install.sh | sh
    os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

    # Clone code
    if not REPO_DIR.exists():
        !git clone https://github.com/linkedlist771/DeMark-World.git {REPO_DIR}

    # Build Venv
    os.chdir(REPO_DIR)
    !uv venv .venv --clear
    # Note: We install mmcv and other heavy deps specifically
    !uv pip install --python .venv/bin/python hatchling uv-dynamic-versioning editables "setuptools<70" wheel build
    !uv sync --no-build-isolation

    # Save Config
    MODELS_DIR = APP_DIR / 'models'
    MODELS_DIR.mkdir(exist_ok=True)
    config = {
        'app_dir': str(APP_DIR), 'repo_dir': str(REPO_DIR),
        'venv_dir': str(REPO_DIR / '.venv'), 'models_dir': str(MODELS_DIR),
        'python': str(REPO_DIR / '.venv' / 'bin' / 'python'),
    }
    (APP_DIR / 'config.json').write_text(json.dumps(config, indent=2))
    MARKER_FILE.touch()

    # --- THE FIX: DEEP INDEXING ---
    print("\n🛰️  Performing Deep Indexing (Hydrating Drive Mount)...")
    # This forces Colab to "see" every folder inside the library path
    lib_path = REPO_DIR / ".venv/lib/python3.12/site-packages"
    !find "{lib_path}" -maxdepth 2 > /dev/null
    os.system('sync')

    print("\n✅ Done! Run Step 2.")

Mounted at /content/drive
🚀 Starting Installation...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
downloading uv 0.10.2 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Cloning into '/content/drive/MyDrive/google-colab/sora-watermark-remover/app/DeMark-World'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 376 (delta 65), reused 357 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (376/376), 12.04 MiB | 15.34 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Updating files: 100% (276/276), done.
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 15 packages in 100ms
Prepared

## Step 2 — Load from Drive *(run every session)*

Mounts Drive and loads everything from the pre-installed environment. **~30 seconds, no downloads.**

In [2]:
#@title ⚡ Step 2: Load Environment { display-mode: "form" }
app_drive_path = "google-colab/sora-watermark-remover" #@param {type:"string"}

import os, sys, json, torch
from pathlib import Path
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive', force_remount=False)

# 2. Define Root
MY_DRIVE = Path('/content/drive/MyDrive')
APP_DIR  = MY_DRIVE / app_drive_path / 'app'
REPO_DIR = APP_DIR / 'DeMark-World'
VENV_DIR = REPO_DIR / '.venv'

# --- LESSON 1: SHELL-LEVEL VERIFICATION ---
print("🔍 Verifying Environment via Shell...")
env_exists = ! [ -f "{APP_DIR}/.install_complete" ] && echo "YES" || echo "NO"
if env_exists[0] == "NO":
    print("❌ Environment not found. Please run Step 1.")
    raise SystemExit()

# --- LESSON 2: PERMISSION RESTORE (CHMOD) ---
print("🔐 Restoring permissions...")
!chmod -R 755 "{VENV_DIR}/bin"

# --- LESSON 3: HYBRID DEPENDENCY BRIDGE ---
print("🔧 Bridging compiled dependencies (ComfyUI style)...")
# We install these locally to ensure they run at full speed and never 'disappear'
!pip install mmcv==2.1.0 loguru ultralytics -q

# 3. Inject Virtual Env paths
site_packages = str(VENV_DIR / 'lib' / 'python3.12' / 'site-packages')
src_path = str(REPO_DIR / 'src')

for p in [site_packages, src_path]:
    if p not in sys.path: sys.path.insert(0, p)

# 4. Environment Variables
os.environ['TORCH_HOME'] = str(APP_DIR / 'models/torch')
os.environ['PATH'] = f"{VENV_DIR}/bin:{os.environ['PATH']}"
os.chdir(str(REPO_DIR))

# 5. Start Engine
try:
    from demark_world.core import DeMarkWorld
    print('\n' + '='*55 + '\n✅ ENGINE LOADED & READY\n' + '='*55)
except Exception as e:
    print(f"❌ Import Error: {e}")
    print("Trying a deep refresh of the site-packages folder...")
    !ls "{site_packages}/demark_world" > /dev/null

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔍 Verifying Environment via Shell...
🔐 Restoring permissions...
🔧 Bridging compiled dependencies (ComfyUI style)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.4/471.4 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 27.6 MB/s eta 0:00:00


KeyboardInterrupt: 

## Step 3 — Auto-Process from Google Drive

Drop videos into your Drive `input/` folder — they'll be processed automatically and saved to `output/`.

| Folder | Purpose |
|---|---|
| `input/` | Drop videos here |
| `output/` | Cleaned videos saved here |
| `processed/` | Originals archived here after processing |

> 💡 LAMA is recommended for T4. Only use E2FGVI_HQ on A100.

In [ ]:
# ==============================================================================
# STEP 3: AUTO-PROCESS (API EDITION - AUTO PATH DETECTION)
# ==============================================================================
#@title 🚀 Launch Auto-Processing (Bypass Drive Lag) { display-mode: "form" }
shared_drive_path = "Figuro/sora-watermark-remover"  #@param {type:"string"}
check_interval_seconds = 10 #@param {type:"slider", min:5, max:60, step:5}
use_lama = True #@param {type:"boolean"}

import os, time, sys, io, threading
from pathlib import Path
from datetime import datetime
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from demark_world.core import DeMarkWorld
from demark_world.schemas import CleanerType

# 1. Authenticate & Build Service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# --- Helper: Find Folder IDs by Path ---
def get_id_for_path(path_str):
    parts = [p for p in path_str.split('/') if p]
    parent_id = 'root'

    # Check for Shared Drives first
    drives_results = drive_service.drives().list(pageSize=100).execute()
    drives = drives_results.get('drives', [])

    # See if the first part of the path is a Shared Drive name
    for d in drives:
        if d['name'] == parts[0]:
            parent_id = d['id']
            parts = parts[1:] # Move to next part of path
            break

    # Traverse folders
    for part in parts:
        query = f"name = '{part}' and '{parent_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
        res = drive_service.files().list(q=query, fields="files(id)", includeItemsFromAllDrives=True, supportsAllDrives=True).execute()
        files = res.get('files', [])
        if not files:
            # Create folder if it doesn't exist
            meta = {'name': part, 'parents': [parent_id], 'mimeType': 'application/vnd.google-apps.folder'}
            new_folder = drive_service.files().create(body=meta, fields='id', supportsAllDrives=True).execute()
            parent_id = new_folder.get('id')
        else:
            parent_id = files[0]['id']
    return parent_id

print("🔍 Resolving Folder IDs from path...")
root_id = get_id_for_path(shared_drive_path)
input_folder_id = get_id_for_path(f"{shared_drive_path}/input")
output_folder_id = get_id_for_path(f"{shared_drive_path}/output")
processed_folder_id = get_id_for_path(f"{shared_drive_path}/processed")

print(f"✅ IDs Resolved:")
print(f"  Root: {root_id}")
print(f"  Input: {input_folder_id}")

# --- Initialization ---
_STOP_EVENT = getattr(sys.modules[__name__], '_STOP_EVENT', None)
if _STOP_EVENT is not None:
    _STOP_EVENT.set()
    time.sleep(2)
_STOP_EVENT = threading.Event()
sys.modules[__name__]._STOP_EVENT = _STOP_EVENT

cleaner_type = CleanerType.LAMA if use_lama else CleanerType.E2FGVI_HQ
demark = DeMarkWorld(cleaner_type=cleaner_type)
SUPPORTED = ('.mp4', '.mov', '.avi', '.mkv', '.webm')
_PROCESSING = set()

# --- API Helper Functions ---
def api_list_files(folder_id):
    query = f"'{folder_id}' in parents and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name)", includeItemsFromAllDrives=True, supportsAllDrives=True).execute()
    return results.get('files', [])

def api_download(file_id, local_path):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()

def api_upload(local_path, filename, folder_id):
    file_metadata = {'name': filename, 'parents': [folder_id]}
    media = MediaFileUpload(local_path, mimetype='video/mp4', resumable=True)
    drive_service.files().create(body=file_metadata, media_body=media, fields='id', supportsAllDrives=True).execute()

def api_move(file_id, old_parent_id, new_parent_id):
    drive_service.files().update(fileId=file_id, addParents=new_parent_id, removeParents=old_parent_id, supportsAllDrives=True).execute()

# --- Core Logic ---
def process_file_api(file_info):
    file_id, filename = file_info['id'], file_info['name']
    if not filename.lower().endswith(SUPPORTED) or file_id in _PROCESSING: return
    _PROCESSING.add(file_id)

    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    local_input = Path('/tmp') / filename
    local_output = Path('/tmp') / f"cleaned_{ts}_{filename}"

    print(f"\n▶ API Processing: {filename}")
    try:
        api_download(file_id, local_input)
        demark.run(local_input, local_output)

        clean_name = f"{ts}_{Path(filename).stem}_cleaned.mp4"
        api_upload(str(local_output), clean_name, output_folder_id)
        api_move(file_id, input_folder_id, processed_folder_id)
        print(f"✅ Success! Moved to processed.")
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        if local_input.exists(): local_input.unlink()
        if local_output.exists(): local_output.unlink()
        _PROCESSING.discard(file_id)

# --- Main Loop ---
print("🤖 Monitor Active (API Mode)...")
try:
    while not _STOP_EVENT.is_set():
        pending = api_list_files(input_folder_id)
        for f in pending:
            if _STOP_EVENT.is_set(): break
            process_file_api(f)
        time.sleep(check_interval_seconds)
except KeyboardInterrupt:
    print("\n🛑 Stopped.")

## Step 4 — Manual: Single Video *(optional)*

Upload one video, process it, download the result. No Drive folder needed.

In [ ]:
#@title 🎬 Manual: Upload & Process Single Video { display-mode: "form" }
use_lama_manual = True  #@param {type:"boolean"}

from pathlib import Path
from google.colab import files
from demark_world.core import DeMarkWorld
from demark_world.schemas import CleanerType
import shutil, os

os.makedirs('resources', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

print('📤 Upload your video:')
uploaded  = files.upload()
filename  = list(uploaded.keys())[0]
input_path = Path('resources') / filename
shutil.move(filename, str(input_path))

cleaner  = CleanerType.LAMA if use_lama_manual else CleanerType.E2FGVI_HQ
label    = 'LAMA' if use_lama_manual else 'E2FGVI_HQ'
out_path = Path('outputs') / f'cleaned_{Path(filename).stem}.mp4'

print(f'\n🚀 Processing with {label}…')
demark = DeMarkWorld(cleaner_type=cleaner)
demark.run(input_path, out_path)

print(f'\n✅ Done → {out_path}')
files.download(str(out_path))


## Step 5 — Manual: Batch Processing *(optional)*

Upload and process multiple videos at once.

In [ ]:
#@title 📦 Manual: Batch Upload & Process { display-mode: "form" }
use_lama_batch = True  #@param {type:"boolean"}

from pathlib import Path
from google.colab import files
from demark_world.core import DeMarkWorld
from demark_world.schemas import CleanerType
from tqdm import tqdm
import shutil

batch_in  = Path('batch_input');  batch_in.mkdir(exist_ok=True)
batch_out = Path('batch_output'); batch_out.mkdir(exist_ok=True)

print('📤 Upload videos:')
uploaded = files.upload()
for fn in uploaded:
    shutil.move(fn, batch_in / fn)
print(f'✅ {len(uploaded)} file(s) ready')

cleaner = CleanerType.LAMA if use_lama_batch else CleanerType.E2FGVI_HQ
label   = 'LAMA' if use_lama_batch else 'E2FGVI_HQ'
videos  = sorted(v for v in batch_in.iterdir() if v.suffix.lower() in ('.mp4','.mov','.avi','.mkv','.webm'))
demark  = DeMarkWorld(cleaner_type=cleaner)

print(f'\n🚀 Processing {len(videos)} video(s) with {label}…\n')
for video in tqdm(videos, desc='Videos'):
    out = batch_out / f'cleaned_{video.name}'
    try:
        demark.run(video, out)
        print(f'  ✅ {video.name}')
    except Exception as e:
        print(f'  ❌ {video.name}: {e}')

print('\n📥 Downloading…')
for f in sorted(batch_out.glob('*.mp4')):
    files.download(str(f))
print('✅ All done!')


## Step 6 — Reinstall / Update *(only if needed)*

Run this if you want to update DeMark-World to the latest version, or if the install got corrupted.

In [ ]:
#@title 🔄 Force Reinstall (MyDrive Only) { display-mode: "form" }
app_drive_path = "google-colab/sora-watermark-remover"  #@param {type:"string"}
import os, shutil
from pathlib import Path

APP_DIR = Path('/content/drive/MyDrive') / app_drive_path / 'app'
MARKER_FILE = APP_DIR / '.install_complete'

if APP_DIR.exists():
    print(f'🗑️ Removing MyDrive app folder: {APP_DIR}')
    if MARKER_FILE.exists(): MARKER_FILE.unlink()
    shutil.rmtree(str(APP_DIR))
    print('✅ Cleaned. Run Step 1 to reinstall.')
else:
    print('Nothing to clean.')


---
## Notes

### Drive Folder Structure
```
Figuro/sora-watermark-remover/
├── app/                    ← entire environment lives here
│   ├── DeMark-World/       ← repo
│   ├── venv/               ← all Python packages
│   ├── models/             ← AI model weights
│   └── config.json         ← paths config
├── input/                  ← drop videos here
├── output/                 ← cleaned videos
└── processed/              ← originals archived after processing
```

### Session Workflow
1. **First time:** Run Step 1 (~5–10 min)
2. **Every session:** Run Step 2 (~30 sec) → then Step 3

### Model Guide
| Model | Speed | VRAM | Best for |
|---|---|---|---|
| LAMA | ~2–5 sec/video-sec | ~5 GB | T4 · fast |
| E2FGVI_HQ | ~10–20 sec/video-sec | ~20 GB | A100 · flicker-free |

### Troubleshooting
- **Step 2 fails** → Run Step 1 first, or Step 6 (reinstall)
- **OOM error** → Switch to LAMA; or *Runtime → Disconnect and delete runtime*
- **Spaces in filename** → Handled automatically via `/tmp` copy
- **Want latest version** → Run Step 6 then Step 1

**Repo:** https://github.com/linkedlist771/DeMark-World